In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
7,Feb. 7: South Dakota reports 98 additional COV...,https://ktiv.com/2021/02/07/feb-7-south-dakota...,"Sun, 07 Feb 2021 17:44:14 GMT",2021-02-07 17:44:14+00:00
1,Vaccines Alone Are Not Enough to Beat COVID S...,https://www.scientificamerican.com/article/vac...,"Sun, 07 Feb 2021 17:00:03 GMT",2021-02-07 17:00:03+00:00
8,Dell Children's part of international study in...,https://www.kxan.com/investigations/dell-child...,"Sun, 07 Feb 2021 17:00:00 GMT",2021-02-07 17:00:00+00:00
6,"10 new COVID-19 deaths, 914 cases reported by ...",https://kstp.com/coronavirus/10-new-covid-19-d...,"Sun, 07 Feb 2021 16:56:40 GMT",2021-02-07 16:56:40+00:00
3,"Virginia COVID-19 Feb. 7 update: Over 2,900 ne...",https://www.wavy.com/news/health/coronavirus/v...,"Sun, 07 Feb 2021 16:02:00 GMT",2021-02-07 16:02:00+00:00
4,"900 new COVID-19 cases, 3 new deaths reported ...",https://www.wlox.com/2021/02/07/new-covid-case...,"Sun, 07 Feb 2021 15:39:59 GMT",2021-02-07 15:39:59+00:00
5,COVID-19 Daily Update 2-7-2021 West Virginia ...,https://dhhr.wv.gov/News/2021/Pages/COVID-19-D...,"Sun, 07 Feb 2021 15:04:42 GMT",2021-02-07 15:04:42+00:00
9,Study finds cognitive problems from COVID-19 ...,https://www.pinejournal.com/newsmd/coronavirus...,"Sun, 07 Feb 2021 13:47:49 GMT",2021-02-07 13:47:49+00:00
0,Covid reinfections may be more common than rea...,https://www.nbcnews.com/health/health-news/cov...,"Sun, 07 Feb 2021 11:00:00 GMT",2021-02-07 11:00:00+00:00
2,"DHEC Director Dr. Edward Simmer, MD, Receives ...",https://scdhec.gov/news-releases/dhec-director...,"Sat, 06 Feb 2021 21:46:47 GMT",2021-02-06 21:46:47+00:00
